Pyro
====


**Pyro** is a **probabilistic programming language** released by Uber AI Labs. It is a tool for deep probabilistic modelling built on top of PyTorch, unifying the best of modern deep learning and Bayesian modeling. [[link]](https://eng.uber.com/pyro/)

A probabilistic programming language is a programming language designed to describe probabilistic models and then perform inference in those models.

Like PyTorch, Pyro supports arbitrary Python code (iteration, recursion, higher-order functions etc.), which makes it universal to represent any computable probability distribution. Thanks to PyTorch, Pyro can benefit from GPU-accelerated tensor math.


In [3]:
!pip install pyro-ppl

In [0]:
# import pytorch and pyro; set random generator seed
import torch

import pyro
import pyro.distributions as dist

import matplotlib.pyplot as plt
from tqdm.notebook import trange

pyro.set_rng_seed(101)

# to clear all variables stored by Pyro
pyro.clear_param_store()

# enable validation for useful warnings and errors
pyro.enable_validation(True)


----

## Primitive Stochastic Functions

The key element of probabilistic programs are primitive stochastic functions, or distributions. Currently, Pyro uses PyTorch distributions: [`torch.distributions`](https://pytorch.org/docs/stable/distributions.html).


In [2]:
# draw a sample from N(0, 1)
loc = 0.  # mean
scale = 1.  # variance
normal_dist = torch.distributions.Normal(loc, scale)  # normal distribution N(0, 1)
x = normal_dist.rsample()  # draw a sample - for N rsample
print(f"normal | sample: {x}")


normal | sample: -0.33274218440055847


In [3]:
# draw a sample from B(alpha=2, beta=5)
alpha = 2
beta = 5
beta_dist = torch.distributions.Beta(alpha, beta)
x = beta_dist.sample()
print(f"beta | sample: {x}")


beta | sample: 0.45079997181892395


### Model

In Pyro, models are Python functions, which take data as input and use Pyro primitives to analyze the data. All probabilistic programs are built with primitive stochastic functions and deterministic computation.

For example, suppose we would like to model daily mean temperatures and cloud cover and how they interact with each other.

Let's declare a function that will describe those interactions:
  *  define a random variable `cloudy`, given by a draw from the Bernoulli distribution with param `0.3`; this will return either `0` or `1`, so then we parse it to a string; according to this line, 30% of the time it's cloudy, 70% it's sunny
  *  define parameters of temperature distributions, i.e. mean and variance, for both cloudy and sunny weather: when it's sunny the mean is 18 degrees Celsius, with variance of 15 degrees, and when it's cloudy the mean equals 13 degrees with variance 15 degrees.
  *  use the values to declare a Normal distribution $\mathcal{N}(\mu, \sigma^2)$ and draw a sample
  *  return the weather and temperature


In [4]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 13.0, 'sunny': 18.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp
    

for _ in range(3):
    print(weather())

('sunny', tensor(53.0301))
('sunny', tensor(15.1199))
('cloudy', tensor(22.8519))


This function is independent of Pyro - it calls PyTorch `torch.distributions`. For a Pyro program we need to apply a few modifications.

## What is the difference between `.sample()` and `.rsample()` - The Reparameterization Trick
Imagine we have the following model and cost function which we want to minimize
$$ x_i \sim \mathcal{N}(\mu, \sigma^2),\  i = 1, \ldots, N, \ x_i \ \mathrm{iid} $$
$$ \mathcal{L}(\mathbb{x}) = \sum_i \left(7 - x_i\right)^2 $$
What are the optimal values for $\mu$ and $\sigma$? Naturally these are 7 and 0 respectively. But imagine we do not know how to solve it straight through and we want to use an iterative technique that utilizes gradients.
But how to calculate $ \frac{\partial \mathcal{L}(\mathbb{x})}{\partial \mu} $ and $ \frac{\partial \mathcal{L}(\mathbb{x})}{\partial \sigma} $ ? At each iteration we sample $N$ data points with $\mathcal{N}(\mu, \sigma^2)$ (we need some initial values for the parameters) and then we loose the track of $\mu$ and $\sigma$. They do not appear in $\mathcal{L}(\mathbb{x})$. What if we reformulate the problem in such a manner?
$$ x_i \sim \mathcal{N}(0, 1),\  i = 1, \ldots, N, \ x_i \ \mathrm{iid} $$
$$ \mathcal{L}(\mathbb{x}) = \sum_i \left(7 - (\sigma x_i + \mu) \right)^2 $$
The models are equivalent, but the gradients become tracktable! This is called `The Reparameterization Trick`. It is widely used in probabilistic machine learning, therefore it is already implemented in `torch`, we do not have to reparametrize manually.
By calling `.sample()` we run the standard sampling process (without reparametrization).
By calling `.rsample()` we run the sampling process with reparametrization - parametrs can be optimized.
Not all distributions can be reparametrized! Check the `has_rsample` property before calling `.rsample()` (or go to documentation).

### `pyro.sample`

It is a Pyro primitive for a random variable. To use `pyro.sample` we need to introduce [`pyro.distributions`](http://docs.pyro.ai/en/1.3.0/distributions.html), which is a thin wrapper for `torch.distributions`, and `pyro.sample`, which replaces all calls of `.sample()` and `.rsample()` (it takes care of reparametrization).


In [5]:
# draw a sample from N(0, 1)
loc = 0.  # mean
scale = 1.  # variance
x = pyro.sample("normal_sample", pyro.distributions.Normal(loc, scale))
print(f"normal | sample: {x}")


normal | sample: -1.3905061483383179


In [6]:
# draw a sample from B(alpha=2, beta=5)
alpha = 2
beta = 5
x = pyro.sample("beta_sample", pyro.distributions.Beta(alpha, beta))
print(f"beta | sample: {x}")


beta | sample: 0.16051988303661346


The crucial difference here is that each sample is **named**. The backend of Pyro uses this names to uniquely identify sample statements and change their behavior at runtime, depending on how the enclosing stochastic function is used. This is how Pyro can implement various manipulations, that underlie inference algorithms.

Here, we use a short name `dist` for `pyro.distributions`.


In [7]:
# weather model rewritten as a Pyro program
def weather():
    cloudy = pyro.sample("cloudy", dist.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 13.0, 'sunny': 18.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample("temp", dist.Normal(mean_temp, scale_temp))
    return cloudy, temp


for _ in range(3):
    print(weather())


('sunny', tensor(34.1101))
('sunny', tensor(35.5346))
('sunny', tensor(34.3806))


Here, `weather()` is still a non-deterministic Python callable, that returns two random samples, but now invoked with `pyro.sample` - because of that Pyro allows us to access joint probability distribution over two named random variables: `cloudy` and `temp`, defining a probabilistic model that we can reason about using the techniques of probabilistic theory.

A natural further step is to build off of the model. For example, ice cream sales depens heavily on the weather. Let's define a situation, where the expected sales depend on the cloudiness and the temperature.


In [0]:
# define ice cream sales as depending on the weather
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == "sunny" and temp > 20.0 else 50.
    ice_cream = pyro.sample("ice_cream", dist.Normal(expected_sales, 10.0))
    return ice_cream

for _ in range(3):
    print(ice_cream_sales())


Since Pyro is embedded in Python, stochastic functions can contain complex deterministic Python - randomness can freely affect control flow! For example, recursive functions could be terminated nondeterministically, provided that we take care to use unique names of `pyro.sample`, such as in this geometric distribution counting the number of failures until the first success. Note, that each `pyro.sample` is created dynamically - we can have different numbers of named random variables.


In [12]:
# count the number of draws from Bernoulli distribution until success
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample(f"x_{t}", dist.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.1))


9


We might also define stochastic functions that accept as input or produce as output other stochastic functions. Here, `normal_product` returns a product of two variables sampled from normal distribution with the same parameters, whereas `make_normal_normal` returns a product of two samples from a normal distribution, where the mean is sampled from $\mathcal{N}(0, 1)$.


In [13]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", dist.Normal(loc, scale))
    z2 = pyro.sample("z2", dist.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", dist.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(normal_product(0, 1))

print(make_normal_normal()(1.))


tensor(-0.1076)
tensor(0.3240)


Excercise
---------


1. Create a Pyro model of flipping a biased coin, using Bernoulli distribution. Set probability as $0.4$. Return the identifier and process in a loop.


In [14]:
def flip_coin():
    return pyro.sample("coin",dist.Bernoulli(0.4))

for _ in range(3):
    print("heads" if flip_coin() == 1 else "tails")


heads
tails
heads


2. Create a Pyro model of how much a person works (hourly employee). By default take Normal distribution with mean = 8, standard deviation = 1. Take into account the following factors:
  *  being sick (decreases the number of hours in a day to 0)
  *  unexpected event (decreases the number of hours by 2)
  *  feeling lazy (leave work one hour earlier)
  *  working overtime (stay two hours longer)
  
  Use Categorical distribution. Assume your own probabilities.


In [0]:
def working_hours():
    

for _ in range(3):
    print(working_hours())